In [1]:
import requests
import datetime
import statsapi
import pandas as pd

In [2]:
# Using mlb-statsapi to get mlb schedule up to the all-star break
# Create dataframe from list genereated from api call
# Creating mapping of mlb team name -> team id

sched = statsapi.schedule(start_date='04/01/2024',end_date='07/16/2024')
sched_df = pd.DataFrame(sched)
teams = sched_df[['away_id', 'away_name']]
teams = teams.drop_duplicates(subset=['away_id'], keep='first')
teams = teams.sort_values(by='away_name', ascending=True)
teams = teams.rename(columns={'away_name': 'team', 'away_id': 'id'})



In [2]:
# MLB Schedule for current date

def print_schedule(schedule):
    """Prints the MLB schedule.

    Args:
        schedule (list): MLB schedule from api call.
    """
    for games in schedule:
        print(f"Date: {games['game_date']} | {games['away_name']} ({games['away_probable_pitcher']}) @ {games["home_name"]} ({games['home_probable_pitcher']})\n")

# {get_pitcher_stats(games['away_probable_pitcher'])}


today = datetime.date.today().strftime('%m/%d/%Y')
mlb_schedule = statsapi.schedule(start_date=today, end_date=today)

print_schedule(mlb_schedule)

Date: 2024-04-10 | Los Angeles Dodgers (Bobby Miller) @ Minnesota Twins (Chris Paddack)

Date: 2024-04-10 | Philadelphia Phillies (Aaron Nola) @ St. Louis Cardinals (Lance Lynn)

Date: 2024-04-10 | Seattle Mariners (Logan Gilbert) @ Toronto Blue Jays (Yusei Kikuchi)

Date: 2024-04-10 | Arizona Diamondbacks (Tommy Henry) @ Colorado Rockies (Austin Gomber)

Date: 2024-04-10 | Washington Nationals (Patrick Corbin) @ San Francisco Giants (Jordan Hicks)

Date: 2024-04-10 | Tampa Bay Rays (Zack Littell) @ Los Angeles Angels (José Soriano)

Date: 2024-04-10 | Chicago White Sox (Erick Fedde) @ Cleveland Guardians (Tanner Bibee)

Date: 2024-04-10 | Chicago Cubs (Kyle Hendricks) @ San Diego Padres (Dylan Cease)

Date: 2024-04-10 | Milwaukee Brewers (Wade Miley) @ Cincinnati Reds (Hunter Greene)

Date: 2024-04-10 | Miami Marlins (Ryan Weathers) @ New York Yankees (Marcus Stroman)

Date: 2024-04-10 | Baltimore Orioles (Cole Irvin) @ Boston Red Sox (Kutter Crawford)

Date: 2024-04-10 | New York Met

In [37]:
import re
import json

# Get pitcher names from MLB schedule
# Store away/home pitcher names into dictionary
# Get pitcher stats and store into another dictionary

def get_pitcher_names(schedule):
    """Gets pitcher names from MLB.

    Args:
        schedule (list): MLB schedule for current day.

    Returns:
        list: Away/Home pitcher names.
    """

    names = []
    for games in schedule:
        names.append({"away": games["away_probable_pitcher"], "home": games["home_probable_pitcher"]})
    return names



def get_pitcher_stats(name):
    """Gets pitcher stats for current year.

    Args:
        name (str): Pitcher full name.

    Returns:
        str: String of all pitcher stats.
    """
    try:
        pitcher = statsapi.player_stats(next(x['id'] for x in statsapi.get('sports_players', {'season':2024})['people'] if x['fullName']==name), 'pitching')
    except StopIteration:
        return ""
        
    return pitcher


def store_pitcher_stats(name, pitcher):
    """Get pitcher stats for current season.

    Args:
        name (str): Pitcher name.
        pitcher (list): List of pitcher stats

    Returns:
        dictionary: for pitcher stats
    """
    
    current_pitcher = {}
    
    stats_pattern = r"(\w+):\s([^\n]+)"
    stats = re.findall(stats_pattern, pitcher)
    
    current_pitcher[name] = dict(stats)
    
    return current_pitcher

def print_pitcher_stats(names):
    
    
    
    raise NotImplementedError

def create_pitcher_df(pitchers):
    pitcher_stats = {}
    for pitcher in pitchers:
        # print(pitcher)
        away = get_pitcher_stats(pitcher["away"])
        home = get_pitcher_stats(pitcher["home"])
    
        away_stats = store_pitcher_stats(pitcher["away"],away)
        home_stats = store_pitcher_stats(pitcher["home"],home)
        # print(f"{away_stats}\t{home_stats}")
        pitcher_stats.update(away_stats)
        pitcher_stats.update(home_stats)
    df = pd.DataFrame(pitcher_stats).transpose()
    df.fillna(0, inplace=True)
    return df

pitcher = statsapi.player_stats(next(x['id'] for x in statsapi.get('sports_players', {'season':2024})['people'] if x['fullName']=='Hunter Greene'), 'pitching')

pitchers = get_pitcher_names(mlb_schedule)
pitcher_df = create_pitcher_df(pitchers)


In [35]:
# https://github.com/toddrob99/MLB-StatsAPI/wiki/Function:-meta
# Get available values from StatsAPI for use in other queries, or look up descriptions for values found in API results.

meta = statsapi.meta('statGroups')


# https://github.com/toddrob99/MLB-StatsAPI/wiki/Function:-notes
# Get notes for a given endpoint.

notes = statsapi.notes('person_stats')
print(notes)

Endpoint: person_stats 
All path parameters: ['ver', 'personId', 'gamePk']. 
Required path parameters (note: ver will be included by default): ['ver', 'personId', 'gamePk']. 
All query parameters: ['fields']. 
Required query parameters: None. 
Developer notes: Specify "current" instead of a gamePk for a player's current game stats.
